In [1]:
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 683.6 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215251 sha256=510df884c54d765c329f0989f46f317c5b4f653457730c179bebbacfa90aef8f
  Stored in directory: /Users/user/Library/Caches/pip/wheels/f0/ae/f9/dfb61e8eab750a5cffaba33a8d1570c6ad0b8bacfd029d3261
Successfully built pyTelegramBotAPI

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


Бот-книга рецептов:</br>
•  Кнопки: "Рецепт дня", "Вегетарианский рецепт", "Поиск рецепта".</br>
•  Функции: Предоставляет различные рецепты.</br>

In [1]:
import pandas as pd
from datetime import datetime, timedelta


data = pd.read_csv("povarenok_recipes_2021_06_16.csv")

data = data.iloc[:data.shape[0] // 2]

data["date"] = pd.date_range(
    start=pd.to_datetime("27/10/2023").strftime("%d/%m/%Y"),
    end=pd.to_datetime((datetime.strptime("27/10/2023", "%d/%m/%Y") + timedelta(days = data.shape[0] - 1)).strftime("%d/%m/%Y")),
)

data

/var/folders/l3/l0vfq_n54s9d8739l34r9qv00000gn/T/ipykernel_5463/4098259681.py:10: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  start=pd.to_datetime("27/10/2023").strftime("%d/%m/%Y"),
/var/folders/l3/l0vfq_n54s9d8739l34r9qv00000gn/T/ipykernel_5463/4098259681.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  end=pd.to_datetime((datetime.strptime("27/10/2023", "%d/%m/%Y") + timedelta(days = data.shape[0] - 1)).strftime("%d/%m/%Y")),


,url,name,ingredients,date
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",2023-10-27
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",2023-10-28
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",2023-10-29
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",2023-10-30
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",2023-10-31
...,...,...,...,...
73286,https://www.povarenok.ru/recipes/show/91669/,Порционный плов с уткой и курагой в тесте фило,"{'Утка': '800 г', 'Рис': '150 г', 'Курага': '3...",2224-06-21
73287,https://www.povarenok.ru/recipes/show/18555/,Суп по-быстрому,"{'Картофель': None, 'Морковь': None, 'Лук репч...",2224-06-22
73288,https://www.povarenok.ru/recipes/show/138887/,Постный фасолевый салат,"{'Фасоль консервированная': '1 бан.', 'Лук реп...",2224-06-23
73289,https://www.povarenok.ru/recipes/show/52377/,"Торт ""Кот в сапогах""","{'Яйцо куриное': '5 шт', 'Сахар': '450 г', 'Ма...",2224-06-24


In [ ]:
import telebot
import config
from telebot import types
from bs4 import BeautifulSoup
from lxml import etree
from urllib import request
import random
import os

token = config.token
bot = telebot.TeleBot(token)


def get_dish_of_the_day():
    date = datetime.now().strftime("%d/%m/%Y")
    sample = data[data["date"] == date]
    name = sample.name.values[0]
    ingredients = ", ".join([f"{key}: {value}" for key, value in eval(sample.ingredients.values[0]).items()])
    
    return name, ingredients


def get_vegan():
    vegan_stopwords = ["баранина",  "говядина", "яицо", "телятина", "цыпленок", "утятина",  "утка", "свинина",  "яйца", "яйцо", "грудка", "крыло", "кур"]
    
    
    def filter_vegeterian(recepie):
        for ingredient in vegan_stopwords:
            if ingredient in recepie:
                return False
        return True
    
    
    found = False
    while not found:
        sample = data.sample(1)
        name = sample.name.values[0]
        ingredients = ", ".join([f"{key}: {value}" for key, value in eval(sample.ingredients.values[0]).items()]).lower()
    
        found = filter_vegeterian(ingredients)
    
    return name, ingredients
        
    
def find_recepie(name):
    data["search"] = data["name"].apply(lambda x: name in x)
    samples = data[data["search"] == True]

    names = list(samples.name.values)
    ingredients = []
    for ingredient_set in samples.ingredients.values:
        ingredients.append(", ".join([f"{key}: {value}" for key, value in eval(ingredient_set).items()]))
    
    return names[:5], ingredients[:5]
    


@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Начать работу")
    markup.add(btn1)
    username = message.from_user.first_name
    bot.send_message(message.chat.id,
                     text=f'Привет, {username}! Я бот, который поможет тебе выбрать блюдо!\n\nДля команд "Рецепт дня" или "Вегетарианский рецепт" нажмите на кнопку\n\nДля команды "Поиск рецепта" напишите Поиск рецепта ваше блюдо',
                     reply_markup=markup)


@bot.message_handler(content_types=['text'])
def func(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Рецепт дня")
    btn2 = types.KeyboardButton("Вегетарианский рецепт")
    back = types.KeyboardButton("Поиск рецепта")
    markup.add(btn1, btn2, back)
    
    if (message.text == "Начать работу"):
        bot.send_message(message.chat.id, text="Что будем делать?", reply_markup=markup)
        
    elif (message.text == 'Рецепт дня'):
        name, ingredients = get_dish_of_the_day()
        bot.send_message(
            message.chat.id, 
            text=f"""
            {name}
            {ingredients}
            """
        )
        
    elif (message.text == "Вегетарианский рецепт"):
        name, ingredients = get_vegan()
        bot.send_message(
            message.chat.id, 
            text=f"""
            {name}
            {ingredients}
            """
        )

    elif message.text.startswith("Поиск рецепта"):
        names, ingredients = find_recepie(message.text[14:])
        
        bot.send_message(message.chat.id, text="\n\n".join([f"{names[i]}\n{ingredients[i]}" for i in range(len(names))]))

        
    else:
        bot.send_message(message.chat.id, text="На такую комманду я не запрограммировал..")


bot.polling(none_stop=True)